In [ ]:
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION  = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/{CUDA_VERSION}/{TORCH_VERSION}/index.html

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

import os
os.kill(os.getpid(), 9)

torch:  2.1 ; cuda:  cu118
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-j4gyzh6c
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-j4gyzh6c
  Resolved https://github.com/facebookresearch/detectron2.git to commit 898507047cf441a1e4be7a729270961c401c4354
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp310-cp310-linux_x86_64.whl size=6118682 sha256=86

In [6]:
import detectron2
import torch, torchvision

import numpy as np
import os, json, cv2, random
from PIL import Image
import matplotlib.pyplot as plt

from detectron2        import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data             import MetadataCatalog, DatasetCatalog
from detectron2.structures       import BoxMode
from detectron2.data.datasets    import register_coco_instances, load_coco_json
from detectron2.utils.logger     import setup_logger
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation       import COCOEvaluator, inference_on_dataset
from detectron2.data             import build_detection_test_loader

import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

import os
import copy
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms, datasets
import torchvision.transforms as T

from torchsummary import summary

from tqdm.notebook import tqdm, trange

import pandas as pd


# from albumentations.pytorch import ToTensorV2
# import albumentations as A

import shutil
from urllib.request import urlretrieve
from torchvision.datasets.utils import download_and_extract_archive

In [7]:
from detectron2.model_zoo.model_zoo import _ModelZooUrls
mz = _ModelZooUrls()
print("\n".join(list(mz.CONFIG_PATH_TO_URL_SUFFIX.keys())))

COCO-Detection/faster_rcnn_R_50_C4_1x
COCO-Detection/faster_rcnn_R_50_DC5_1x
COCO-Detection/faster_rcnn_R_50_FPN_1x
COCO-Detection/faster_rcnn_R_50_C4_3x
COCO-Detection/faster_rcnn_R_50_DC5_3x
COCO-Detection/faster_rcnn_R_50_FPN_3x
COCO-Detection/faster_rcnn_R_101_C4_3x
COCO-Detection/faster_rcnn_R_101_DC5_3x
COCO-Detection/faster_rcnn_R_101_FPN_3x
COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x
COCO-Detection/retinanet_R_50_FPN_1x
COCO-Detection/retinanet_R_50_FPN_3x
COCO-Detection/retinanet_R_101_FPN_3x
COCO-Detection/rpn_R_50_C4_1x
COCO-Detection/rpn_R_50_FPN_1x
COCO-Detection/fast_rcnn_R_50_FPN_1x
COCO-InstanceSegmentation/mask_rcnn_R_50_C4_1x
COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_1x
COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x
COCO-InstanceSegmentation/mask_rcnn_R_50_C4_3x
COCO-InstanceSegmentation/mask_rcnn_R_50_DC5_3x
COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x
COCO-InstanceSegmentation/mask_rcnn_R_101_C4_3x
COCO-InstanceSegmentation/mask_rcnn_R_101_DC5_3x
COCO-Insta

In [8]:
!git clone https://github.com/AAEfimov/PP.git

dataset_directory = "PP"
content = os.listdir(dataset_directory)
print(content)

fatal: destination path 'PP' already exists and is not an empty directory.
['train', 'README.md', '.git']


In [9]:

dataset_catalog_name = "stone_train"

try:
    DatasetCatalog.pop(dataset_catalog_name)
except:
    pass


convert json to spec format

https://detectron2.readthedocs.io/en/latest/tutorials/datasets.html

Enum of different ways to represent a box.

XYXY_ABS= 0
XYWH_ABS= 1
XYXY_REL= 2
XYWH_REL= 3
XYWHA_ABS= 4

In [25]:

def get_stone_dict(fp):
  fn = fp + "dataset.json"
  with open(fn) as f:
    data = json.load(f)

  dataset_dicts = []

  for img in data['images']:
    record = {}

    filename = os.path.join(fp, img["file_name"])

    record["file_name"] = filename
    record["image_id"] = img['id']
    record["height"] = img['height']
    record["width"] = img['width']

    objs = []
    for anno in data['annotations']:
      if anno['image_id'] == record["image_id"]:
        obj = {
            "bbox" : anno['bbox'],
            "bbox_mode": BoxMode.XYWH_ABS,
            # "bbox_mode" : BoxMode.XYXY_ABS,
            "segmentation": anno['segmentation'],
            "category_id": 0,
            # anno['category_id'],
            "iscrowd": 0,

        }

        objs.append(obj)

    record["annotations"] = objs
    dataset_dicts.append(record)

  return dataset_dicts


In [28]:
# traint_dataset_pwd = dataset_directory + "/train/"
# print(get_stone_dict(traint_dataset_pwd))


traint_dataset_pwd = dataset_directory + "/train/"

try:
    DatasetCatalog.register(dataset_catalog_name, lambda : get_stone_dict(traint_dataset_pwd))
except:
    print(f'Probably data {traint_dataset_pwd} have been already registred')

MetadataCatalog.get(dataset_catalog_name).set(thing_classes=["stone"])

stone_metadata = MetadataCatalog.get("stone_train")
dataset_dicts    = get_stone_dict(traint_dataset_pwd)
stone_metadata.as_dict()
print('N Train',len(dataset_dicts))
print(dataset_dicts)

Probably data PP/train/ have been already registred
N Train 12
[{'file_name': 'PP/train/11-11-2021-12-25-32_1_1_63_detailed_part7.bmp', 'image_id': 1, 'height': 2048, 'width': 2592, 'annotations': [{'bbox': [0.0, 63.97872340425532, 501.08510638297867, 1036.1702127659573], 'bbox_mode': <BoxMode.XYWH_ABS: 1>, 'segmentation': [[11, 1100, 13, 1010, 90, 949, 167, 915, 237, 798, 377, 772, 447, 578, 501, 406, 496, 310, 435, 251, 392, 191, 279, 174, 158, 189, 167, 132, 120, 63, 56, 110, 18, 178, 0, 222]], 'category_id': 0, 'iscrowd': 0}, {'bbox': [0.0, 1281.0, 303.21276595744666, 470.0], 'bbox_mode': <BoxMode.XYWH_ABS: 1>, 'segmentation': [[13, 1281, 107, 1359, 181, 1502, 224, 1627, 303, 1685, 288, 1742, 164, 1715, 35, 1740, 0, 1751]], 'category_id': 0, 'iscrowd': 0}, {'bbox': [5.340425531914889, 755.4680851063829, 1085.1063829787233, 912.7659574468084], 'bbox_mode': <BoxMode.XYWH_ABS: 1>, 'segmentation': [[5, 1191, 64, 1046, 162, 972, 226, 834, 320, 783, 458, 783, 643, 755, 718, 763, 803, 853

In [29]:
for d in random.sample(dataset_dicts, 2):
    img   = Image.open(d["file_name"])
    _,axs = plt.subplots(1,2,figsize=(12,8))
    axs[0].imshow(img); axs[0].axis('off')
    visualizer = Visualizer(img, metadata=stone_metadata, scale=1)
    out = visualizer.draw_dataset_dict(d)
    axs[1].imshow(out.get_image()); axs[1].axis('off')
    plt.tight_layout();plt.show()

Output hidden; open in https://colab.research.google.com to view.

Add trainer

In [32]:
from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)


In [33]:
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
import copy
import torch

def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below

    image = utils.read_image(dataset_dict["file_name"], format="BGR")

    transform_list = [
                        T.Resize((800,600)),
                        T.RandomBrightness(0.8, 1.8),
                        T.RandomContrast(0.6, 1.3),
                        T.RandomSaturation(0.8, 1.4),
                        T.RandomRotation(angle=[90, 90]),
                        T.RandomLighting(0.7),
                        T.RandomFlip(prob=0.4, horizontal=False, vertical=True),
                    ]

    image, transforms = T.apply_transform_gens(transform_list, image)

    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]

    instances = utils.annotations_to_instances(annos, image.shape[:2])

    dataset_dict["instances"] = utils.filter_empty_instances(instances)

    return dataset_dict



In [40]:
BATCH = 2

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("stone_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = BATCH
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)

In [41]:
import torch.multiprocessing as mp
mp.set_start_method('spawn', force=True)

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True);
trainer = CustomTrainer(cfg)#DefaultTrainer(cfg);

trainer.resume_or_load(resume=False)

[10/31 08:45:22 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


In [42]:
trainer.train()

[10/31 08:45:27 d2.engine.train_loop]: Starting training from iteration 0
[10/31 08:45:42 d2.utils.events]:  eta: 0:02:21  iter: 19  total_loss: 3.772  loss_cls: 0.733  loss_box_reg: 0.9583  loss_mask: 0.6892  loss_rpn_cls: 1.409  loss_rpn_loc: 0.1233    time: 0.5140  last_time: 0.4286  data_time: 0.4299  last_data_time: 0.0653   lr: 1.6068e-05  max_mem: 2129M
[10/31 08:45:54 d2.utils.events]:  eta: 0:02:10  iter: 39  total_loss: 2.569  loss_cls: 0.6279  loss_box_reg: 0.9455  loss_mask: 0.6824  loss_rpn_cls: 0.2337  loss_rpn_loc: 0.106    time: 0.5504  last_time: 0.4497  data_time: 0.1975  last_data_time: 0.1044   lr: 3.2718e-05  max_mem: 2130M
[10/31 08:46:07 d2.utils.events]:  eta: 0:02:04  iter: 59  total_loss: 2.323  loss_cls: 0.5605  loss_box_reg: 0.928  loss_mask: 0.6625  loss_rpn_cls: 0.07054  loss_rpn_loc: 0.08857    time: 0.5895  last_time: 0.4541  data_time: 0.2925  last_data_time: 0.1430   lr: 4.9367e-05  max_mem: 2130M
[10/31 08:46:21 d2.utils.events]:  eta: 0:01:56  iter: 

In [54]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("stone_val", )
predictor = DefaultPredictor(cfg)

from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

[10/31 09:02:36 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


In [52]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"



events.out.tfevents.1698741508.656556ec484c.6101.0  last_checkpoint  model_final.pth
events.out.tfevents.1698741942.656556ec484c.6101.1  metrics.json
335M	output
